<a href="https://colab.research.google.com/github/aybberrada/youtube-whisper-colab/blob/main/Youtube_Whisperer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title Setup
!pip install git+https://github.com/openai/whisper.git
!pip install pytube

import whisper
from pytube import YouTube
import sys
import os
from google.colab import files
from IPython.display import HTML as html_print
from datetime import timedelta

In [7]:
#@title Run
Video_URL = "https://www.youtube.com/watch?v=Sz1n0RHwLqA" #@param {type:"string"}
Video_in_English = True #@param {type:"boolean"}
model = 'tiny' #@param ["tiny", "base", "small", "medium"]
output_format = 'srt' #@param ["text", "srt"]

#@markdown Expected time depending on video length and model (quality, speed):
#@markdown
#@markdown       base: ( Better, Fast )                tiny: ( Very good, Fastest)
#@markdown            ~2 hours      in 5 minutes     |        ~2 hours      in 3 minutes
#@markdown            ~1 hour       in 4 minutes     |        ~1 hour       in 2 minutes
#@markdown            ~30 minutes   in 2 minutes     |        ~30 minutes or less  in less than 1 minute
#@markdown
#@markdown       small: ( 2nd Best, Slow  )            medium: ( Best, Slowest )
#@markdown            ~2 hours      in 9 minutes    |        ~2 hours      in 28 minutes
#@markdown            ~1 hour       in 7 minutes    |        ~1 hour       in 21 minutes
#@markdown            ~30 minutes   in 4 minutes     |        ~30 minutes   in 15 minutes
#@markdown

print("Loading Whisper...")
model = model+".en" if Video_in_English else model
w_model = whisper.load_model(model)

print("Downloading audio...")
youtube_video = YouTube(Video_URL)
audio_stream = youtube_video.streams.filter(only_audio=True).first()
audio_stream.download(filename='audio.mp4')

print("Transcribing audio...")
if Video_in_English:
  output = w_model.transcribe('./audio.mp4', language="en")
else:
  output = w_model.transcribe('./audio.mp4')

if output_format == "text":
  with open("transcription.txt","w") as f:
    f.write(output['text'])
    f.close()
  print("Done. Please wait...")
  files.download("transcription.txt")

elif output_format == "srt":
    segments = output['segments']
    for segment in segments:
        startTime = str(0)+str(timedelta(seconds=int(segment['start'])))+',000'
        endTime = str(0)+str(timedelta(seconds=int(segment['end'])))+',000'
        text = segment['text']
        segmentId = segment['id']+1
        segment = f"{segmentId}\n{startTime} --> {endTime}\n{text[1:] if text[0] == ' ' else text}\n\n"

        with open("subtitles.srt", 'a', encoding='utf-8') as srtFile:
            srtFile.write(segment)
    print("Done. Please wait...")
    files.download("subtitles.srt")

Loading Whisper...
Transcribing audio...
Done. Please wait...


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>